This notebook shows how you could split Fashion outfits file in order to have train and test datasets to experiment with your model.

**Disclaimer**: The process used here is alway simple than the one we used to generate the test files for the leaderboard, but you could expect that some tricks used here (like dealing with products as a set, or generating candidates based on entire product_id list) are also used on the leaderboard files.

This code takes about 40 minutes to run in a common notebook

In [ ]:
import datetime
print(datetime.datetime.now())

In [ ]:
import pandas as pd

full_outfits = pd.read_parquet("../data/manual_outfits.parquet")
full_outfits.head()

To generate more examples given an particular outfit, one way is shuffing the products, remember that outfits are sets of products. This step is here just to ilustrate this possibility.

In [ ]:
from numpy.random import permutation

full_outfits["products_shuffled"] = full_outfits.apply(lambda row: permutation(row["products"]).tolist(), axis=1)
full_outfits.head()

To generate a train example to use in your model, you could split a outfit in incomplete_outfit and missing_product.

In [ ]:
full_outfits["incomplete_outfit"] = full_outfits.apply(lambda row: row["products_shuffled"][:-1], axis=1)
full_outfits["missing_product"] = full_outfits.apply(lambda row: row["products_shuffled"][-1], axis=1)
full_outfits.head()

In [ ]:
items_metadata = pd.read_parquet("../data/products.parquet")
items_metadata = items_metadata["product_id"]
items_metadata.head()

One way to generate candidates is to sample then from all product_id in the dataset.

In [ ]:
from random import randint

def candidates(row, minc=8, maxc=40):
    n = randint(minc, maxc)
    c = items_metadata.sample(n).unique().tolist()
    c.append(row["missing_product"])
    return list(set(c))

full_outfits["candidates"] = full_outfits.apply(lambda row: candidates(row), axis=1)
full_outfits.head()

A example of split on 80% for training and 20% for test

In [ ]:
train = full_outfits.sample(frac=0.8)
train.head()

In [ ]:
test = full_outfits[~full_outfits.outfit_id.isin(set(train["outfit_id"].values.tolist()))]
test.head()

In [ ]:
test_input = test[["outfit_id", "incomplete_outfit", "candidates"]]
test_input.head()

In [ ]:
test_output = test[["outfit_id", "missing_product"]]
test_output.head()

In [ ]:
import time

unique_name = int(time.time())
train.to_parquet(f"../data/manual_outfits_train_{unique_name}.parquet")
test_input.to_parquet(f"../data/manual_outfits_testinput_{unique_name}.parquet")
test_output.to_csv(f"../data/manual_outfits_testoutput_{unique_name}.csv", index=False)

In [ ]:
print(datetime.datetime.now())